In [1]:
## Import the library
from __future__ import division
import sys
import os
#os.chdir('../data/MINER_MicroLowessRNATMM.08.24.2020/')
import miner as miner
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns
import time
import imp
imp.reload(miner)
import miner_py3_kk as miner3
from lifelines import KaplanMeierFitter# Path to the miner directory
from tqdm.notebook import tqdm, trange

# Miner Output folder
input_path = "/Volumes/omics4tb2/SYGNAL/GBM-Serdar/MINER_MicroLowessRNATMM.08.24.2020"
print("Done!")

Done!


# Load data files

In [2]:
# load Gene ID mapping file and filter for Gene Name
identifiers = pd.read_csv("../data/identifier_mappings.txt", header=0, index_col=None, sep="\t")
identifiers = identifiers[identifiers['Source'] == 'Gene Name']
identifiers = identifiers[['Preferred_Name','Name','Source']]

# load master OT drug table
drugTargetCombo = pd.read_csv("../data/opentargets_gbm_longer.csv", header=0, index_col=None,encoding= 'unicode_escape')
drugTargetCombo = drugTargetCombo.rename({'molecule_name':'Drug'}, axis=1)
drugTargetCombo = drugTargetCombo.rename({'approved_name':'TargetSymbol'}, axis=1)
print("Loaded " + str(len(drugTargetCombo['Drug'].drop_duplicates())), " drugs.")

# # load MINER Regulons and add Regulator and Gene Symbols
regulonDf1 = pd.read_csv(os.path.join(input_path, "regulonDf.csv"), header = 0,index_col=0)
regulonDf2 = pd.merge(regulonDf1, identifiers, left_on = ['Gene'], right_on = ['Preferred_Name'], how="left")
regulonDf2.columns = ['Regulon_ID','Regulator','Gene','Gene_ID','GeneSymbol','Source']
regulonDf = pd.merge(regulonDf2, identifiers, left_on = ['Regulator'], right_on = ['Preferred_Name'], how="left")
regulonDf.columns = ['Regulon_ID','Regulator','Gene','Gene_ID','GeneSymbol','Source_x','Regulator_ID','RegulatorSymbol','Source_y']
print("Loaded " + str(len(regulonDf.drop_duplicates())), " regulons.")

# Load CM Flows
cmFlows = pd.read_csv(os.path.join(input_path, "filteredCausalResults_miRNAAndTFMinerGbmMicroarrayRNASeq.csv"), header = 0)
print("Loaded " + str(len(cmFlows['Regulon_ID'])), " CM Flows.")

Loaded 3218  drugs.
Loaded 45908  regulons.
Loaded 39236  CM Flows.


# Drug to CM flow mapping

In [4]:
ResultsList = []

drugs = list(drugTargetCombo['Drug'].drop_duplicates())
#drugs = ['AZD-4547']
for i in tqdm(drugs):
    time.sleep(0.01)
    #print(i)

    targetslist = list(drugTargetCombo[drugTargetCombo['Drug']==i]['TargetSymbol'].drop_duplicates())
    #targetslist = targetslist[0].split(':')
    #print(targetslist)
    #regulonDf = pd.read_csv(os.path.join(input_path, "regulonDf.csv"), header = 0)

    #loop  through every target symbol for every drug
    for j in targetslist:
        #print(j)
        if j in regulonDf.GeneSymbol.values:
            #print(j + ' is in Regulon Genes')
            regulonids = regulonDf[regulonDf['GeneSymbol']==j]['Regulon_ID']
            regulonids = list(regulonids.drop_duplicates())
            for k in regulonids:
                ResultsList.append([i, j, k, 'Regulon_Gene'])

        if j in cmFlows.MutationGene.values:
            #print(j + ' is in Mutations')
            mutations = cmFlows[cmFlows['MutationGene']==j]['Regulon_ID']
            mutations = list(mutations.drop_duplicates())
            for l in mutations:
                ResultsList.append([i, j, l, 'Mutation_Gene'])

        if j in cmFlows.RegulatorSymbol.values:
            #print(j + ' is in Regulators')
            regulators = cmFlows[cmFlows['RegulatorSymbol']==j]['Regulon_ID']
            regulators = list(regulators.drop_duplicates())
            for m in regulators:
                ResultsList.append([i, j, m, 'Regulator_Gene'])                     
                
                
#Convert list to dataframe
ResultsListDf = pd.DataFrame(ResultsList)
#remove duplicates from dataframe
ResultsListDf = ResultsListDf.drop_duplicates()
#assign columns to dataframe
ResultsListDf.columns = ['Drug', 'TargetSymbol', 'Regulon_ID', 'Miner_Target_Type']
#ResultsListDf = ResultsListDf[['Drug', 'GeneSymbol', 'Regulon_ID']]
ResultsListDf = ResultsListDf.drop_duplicates()
print("1: ",ResultsListDf.shape)

# merge with other drug info
ResultsListDf = pd.merge(ResultsListDf, drugTargetCombo, left_on = ['Drug','TargetSymbol'], right_on = ['Drug','TargetSymbol'], how="left")
print("2: ",ResultsListDf.shape)

resultsRegulonGenes = ResultsListDf[ResultsListDf['Miner_Target_Type'] == 'Regulon_Gene']
resultsMutationGenes = ResultsListDf[ResultsListDf['Miner_Target_Type'] == 'Mutation_Gene']
resultsRegulatorGenes = ResultsListDf[ResultsListDf['Miner_Target_Type'] == 'Regulator_Gene']

resultsRegulonGenesDf = pd.merge(resultsRegulonGenes, cmFlows, left_on = ['Regulon_ID'], right_on = ['Regulon_ID'], how="left")
resultsMutationGenesDf = pd.merge(resultsMutationGenes, cmFlows, left_on = ['Regulon_ID','TargetSymbol'], right_on = ['Regulon_ID','MutationGene'], how="left")
resultsRegulatorGenesDf = pd.merge(resultsRegulatorGenes, cmFlows, left_on = ['Regulon_ID','TargetSymbol'], right_on = ['Regulon_ID','RegulatorSymbol'], how="left")

print("3: RegulonGenes ",resultsRegulonGenes.shape, resultsRegulonGenesDf.shape )
print("4: MutationGenes ",resultsMutationGenes.shape, resultsMutationGenesDf.shape )
print("5: RegulatorGenes ",resultsRegulatorGenes.shape, resultsRegulatorGenesDf.shape )

# merge with regulator info
#ResultsListDf['Regulon_ID'] = ResultsListDf['Regulon_ID'].astype(int)
#print(ResultsListDf)
# get regulators from regulonDf
#regulonDfRegulators = regulonDf[['Regulon_ID', 'Regulator']].drop_duplicates()

#ResultsListDf = pd.merge(ResultsListDf, regulonDfRegulators, on = ['Regulon_ID'], how="left")
#print(ResultsListDf)

#cmFlows['Regulon_ID'] = cmFlows['Regulon_ID'].astype(int)

#ResultsListDf = pd.merge(ResultsListDf, cmFlows, left_on = ['Regulon_ID','GeneSymbol'], right_on = ['Regulon_ID.1'], how="left")
#print(ResultsListDf)
ResultsListDfAll = pd.concat([resultsRegulonGenesDf,resultsMutationGenesDf,resultsRegulatorGenesDf])
#print("3 ",ResultsListDf)
print("6: ResultsListDfAll ",ResultsListDfAll.shape)


  0%|          | 0/3218 [00:00<?, ?it/s]

1:  (77139, 4)


KeyError: 'GeneSymbol'

In [6]:
ResultsListDfAll.to_csv("../data/GBM_Master_Drugs_Mapped_CMFlows_120921.csv",index=0)


In [5]:

# merge with other drug info
ResultsListDf = pd.merge(ResultsListDf, drugTargetCombo, left_on = ['Drug','TargetSymbol'], right_on = ['Drug','TargetSymbol'], how="left")
print("2: ",ResultsListDf.shape)

resultsRegulonGenes = ResultsListDf[ResultsListDf['Miner_Target_Type'] == 'Regulon_Gene']
resultsMutationGenes = ResultsListDf[ResultsListDf['Miner_Target_Type'] == 'Mutation_Gene']
resultsRegulatorGenes = ResultsListDf[ResultsListDf['Miner_Target_Type'] == 'Regulator_Gene']

resultsRegulonGenesDf = pd.merge(resultsRegulonGenes, cmFlows, left_on = ['Regulon_ID'], right_on = ['Regulon_ID'], how="left")
resultsMutationGenesDf = pd.merge(resultsMutationGenes, cmFlows, left_on = ['Regulon_ID','TargetSymbol'], right_on = ['Regulon_ID','MutationGene'], how="left")
resultsRegulatorGenesDf = pd.merge(resultsRegulatorGenes, cmFlows, left_on = ['Regulon_ID','TargetSymbol'], right_on = ['Regulon_ID','RegulatorSymbol'], how="left")

print("3: RegulonGenes ",resultsRegulonGenes.shape, resultsRegulonGenesDf.shape )
print("4: MutationGenes ",resultsMutationGenes.shape, resultsMutationGenesDf.shape )
print("5: RegulatorGenes ",resultsRegulatorGenes.shape, resultsRegulatorGenesDf.shape )

# merge with regulator info
#ResultsListDf['Regulon_ID'] = ResultsListDf['Regulon_ID'].astype(int)
#print(ResultsListDf)
# get regulators from regulonDf
#regulonDfRegulators = regulonDf[['Regulon_ID', 'Regulator']].drop_duplicates()

#ResultsListDf = pd.merge(ResultsListDf, regulonDfRegulators, on = ['Regulon_ID'], how="left")
#print(ResultsListDf)

#cmFlows['Regulon_ID'] = cmFlows['Regulon_ID'].astype(int)

#ResultsListDf = pd.merge(ResultsListDf, cmFlows, left_on = ['Regulon_ID','GeneSymbol'], right_on = ['Regulon_ID.1'], how="left")
#print(ResultsListDf)
ResultsListDfAll = pd.concat([resultsRegulonGenesDf,resultsMutationGenesDf,resultsRegulatorGenesDf])
#print("3 ",ResultsListDf)
print("6: ResultsListDfAll ",ResultsListDfAll.shape)


2:  (77139, 20)
3: RegulonGenes  (31394, 20) (31394, 34)
4: MutationGenes  (43019, 20) (43172, 34)
5: RegulatorGenes  (2726, 20) (35587, 34)
6: ResultsListDfAll  (110153, 34)
